# Sampling Techniques using KerasNLP

**Author:** [Usha Rengaraju](https://www.linkedin.com/in/usha-rengaraju-b570b7a2/)<br>
**Date created:** 2023/07/10<br>
**Last modified:** 2023/07/10<br>
**Description:** Discussion of various sampling techniques using KerasNLP

## Overview

KerasNLP offers a wide range of sampling techniques already implemented and very easy to use. Samplers are basically the methods of selecting the next token based on the distribution of the previous tokens. There are various sampling techniques like `Beam Sampler` , `Contrastive Sampler`, `Random Sampler` and many more.

This guide will show you how to use the various samplers and will also show the various hyperparameters of these samplers and how the output will be affected by these.


## Imports & setup

This tutorial requires you to have KerasNLP installed:

```shell
pip install keras-nlp
```

We begin by importing all required packages:

## Model Loading

For this tutorial we will be using a pretrained `OPT Language Model` which is also directed loaded from the kerasnlp.

For the output generation we take the sample input as `I recently ate`.

In [ ]:
import keras_nlp

opt_lm = keras_nlp.models.OPTCausalLM.from_preset("opt_1.3b_en")

5263448608/5263448608 [==============================] - 265s 0us/step


In [ ]:
opt_lm.generate("I recently ate", max_length=50)

"I recently ate some raw mango and I had to stop because I was getting so sick. I had to go to the bathroom so much. It was awful. It was so bad, I was throwing up. I'm glad I was alone because"

#Beam Sampler

At each time-step, beam search keeps the beams (sequences) of the top num_beams highest accumulated probabilities, and uses each one of the beams to predict candidate next tokens.

### setting num_beams=2


In [ ]:
opt_lm.compile(sampler=keras_nlp.samplers.BeamSampler(num_beams=2))
opt_lm.generate("I recently ate", max_length=50)

'I recently ate at a restaurant that had a sign on the wall that said "no dogs allowed".   I asked the waitress if I could bring my dog. She said no. I asked if I could bring my cat. She said no'

### num_beams=10

In [ ]:
opt_lm.compile(sampler=keras_nlp.samplers.BeamSampler(num_beams=10))
opt_lm.generate("I recently ate", max_length=50)

"I recently ate there for the first time. It was pretty good, but I don't think I'll be going back anytime soon. The service was terrible, the food was mediocre at best, and the drinks were overpriced.   I"

### num_beams=100


In [ ]:
opt_lm.compile(sampler=keras_nlp.samplers.BeamSampler(num_beams=100))
opt_lm.generate("I recently ate", max_length=50)

"I recently ate there for the first time, and I have to say it was one of the best meals I've had in a long time.                    "

#Greedy Sampler

This sampler is implemented on greedy search, i.e., always picking up the token of the largest probability as the next token.

In [ ]:
opt_lm.compile(sampler=keras_nlp.samplers.GreedySampler())
opt_lm.generate("I recently ate", max_length=50)

"I recently ate a whole bag of these. I was so disappointed.\nI'm sorry to hear that. I've had them a few times and they're pretty good."

#Contrastive Sampler

This sampler implements contrastive search algorithm. In short, the sampler chooses the token having the max "score" as the next token. The "score" is a weighted sum between token's probability and max similarity against previous tokens. By using this joint score, contrastive sampler reduces the behavior of duplicating seen tokens.

**k**: int, the k value of top-k. Next token will be chosen from k tokens.<br>
**alpha**: float, the weight of minus max similarity in joint score computation. The larger the value of alpha, the score relies more on the similarity than the token probability.<br>
**seed**: int, defaults to None. The random seed.

### Large alpha small k

In [ ]:
opt_lm.compile(sampler=keras_nlp.samplers.ContrastiveSampler(k=2, alpha=0.9, seed=45))
opt_lm.generate("I recently ate", max_length=50)

'I recently ate at a restaurant that had an all you can eat buffet. I was the first person in line and the server asked if I wanted to try anything. I said no, but I was curious about what they had to offer. She'

### Large alpha large k

In [ ]:
opt_lm.compile(sampler=keras_nlp.samplers.ContrastiveSampler(k=10, alpha=0.9, seed=45))
opt_lm.generate("I recently ate", max_length=50)

'I recently ate my weight (well a ton). It‼\u200d♀️\nCongo is proud to receive such well thought food. You should feel better for that, atleaet :)) Have my uptoke to counter'

### small alpha large k

In [ ]:
opt_lm.compile(sampler=keras_nlp.samplers.ContrastiveSampler(k=2, alpha=0.1, seed=45))
opt_lm.generate("I recently ate", max_length=50)

"I recently ate a whole bag of Doritos and a box of Cheetos. I'm not sure if I should be proud or ashamed.\nI'm proud of you."

### Small alpha small k

In [ ]:
opt_lm.compile(sampler=keras_nlp.samplers.ContrastiveSampler(k=2, alpha=0.1, seed=45))
opt_lm.generate("I recently ate", max_length=50)

"I recently ate a whole bag of Doritos and a box of Cheetos. I'm not sure if I should be proud or ashamed.\nI'm proud of you."

### Large k

In [ ]:
opt_lm.compile(sampler=keras_nlp.samplers.ContrastiveSampler(k=20, alpha=0.6, seed=45))
opt_lm.generate("I recently ate", max_length=50)

'I recently ate at Pangea Seafood (they’re on Stoughton). Service was great, I didn’t feel like I was intruding on their day to day so I sat next to the cashier and ch'

### Small k


In [ ]:
opt_lm.compile(sampler=keras_nlp.samplers.ContrastiveSampler(k=2, alpha=0.6, seed=45))
opt_lm.generate("I recently ate", max_length=50)

'I recently ate at a restaurant that had a "sausage and egg" sandwich. It was a sausage, egg, and cheese sandwich with a side of bacon. I don\'t know what the hell they were thinking, but it was delicious'

#Random Sampler

This sampler implements random sampling. Briefly, random sampler randomly selects a token from the entire distribution of the tokens, with selection chance determined by the probability of each token.

**seed**: int, defaults to None. The random seed.

In [ ]:
opt_lm.compile(sampler=keras_nlp.samplers.RandomSampler(seed=51))
opt_lm.generate("I recently ate", max_length=50)

"I recently ate chiethaya soup. My whole face seemed to have watery eyes for like the whole day. If you're in Bangkok I had no experience with it, it's one of the pearl shops and you try 3 items before purchasing"

#Top K Sampler

This sampler implements top-k search algorithm. Briefly, top-k algorithm randomly selects a token from the tokens of top K probability, with selection chance determined by the probability.

**k**: int, the k value of top-k. Next token will be chosen from k tokens.<br>
**seed**: int, defaults to None. The random seed.

### k=2

In [ ]:
opt_lm.compile(sampler=keras_nlp.samplers.TopKSampler(k=5, seed=30))
opt_lm.generate("I recently ate", max_length=50)

"I recently ate an egg that had been sitting out for a week, it wasn't as bad as the first egg I ate but still gross.\nI think that's just the first one.\nI don't even know what to think anymore"

### k=10

In [ ]:
opt_lm.compile(sampler=keras_nlp.samplers.TopKSampler(k=10, seed=30))
opt_lm.generate("I recently ate", max_length=50)

'I recently ate an entire can of tuna with some chips. I\'m a pretty skinny dude, but this was a new level of "I\'m not fat."\nI\'ve been eating tuna for breakfast for several months now, because it\'s easy'

### k=100

In [ ]:
opt_lm.compile(sampler=keras_nlp.samplers.TopKSampler(k=100, seed=30))
opt_lm.generate("I recently ate", max_length=50)

"I recently ate my first chicken sandwich from Piedmont Fresh Market. Not a huge fan because the tomatoes are a bit watery and the breading isn't very chewy. The tomatoes made the sandwich rather sweet and sour.   I"

#Top-P Sampler

This sampler implements top-p search algorithm. Top-p search selects tokens from the smallest subset of output probabilities that sum to greater than p. Put in another way, top-p will first order token predictions by likelihood, and ignore all tokens after the cumulative probability of selected tokens exceeds p, then select a token from the remaining tokens.
<br>
<br>
**p**: float, the p value of top-p.<br>
**k**: int, defaults to None. If set, this argument defines a heuristic "top-k" cutoff applied before the "top-p" sampling. All logits not in the top k will be discarded, and the remaining logits will be sorted to find a cutoff point for p. Setting this arg can significantly speed sampling up by reducing the number of tokens to sort.<br>
**seed**: int, defaults to None. The random seed.

### Low p and no top-k

In [ ]:
opt_lm.compile(sampler=keras_nlp.samplers.TopPSampler(p=0.1, k=None, seed=22))
opt_lm.generate("I recently ate", max_length=50)

"I recently ate a lot of meat and I'm not sure if I'm going to be able to stop. I'm not sure if I'm going to be able to stop.\nI'm not sure if I'm going to be able to"

### Low p and top-k

In [ ]:
opt_lm.compile(sampler=keras_nlp.samplers.TopPSampler(p=0.1, k=5, seed=22))
opt_lm.generate("I recently ate", max_length=50)

"I recently ate a lot of spicy food and I'm still feeling it. I'm not sure if it's the heat or the fact that I'm still recovering from the spicy food.\nI'm the same way. I'm not sure if"

### High p and no top-k

In [ ]:
opt_lm.compile(sampler=keras_nlp.samplers.TopPSampler(p=0.8, k=None, seed=22))
opt_lm.generate("I recently ate", max_length=50)

'I recently ate an entire bag of Oatmeal Raisin cookies in one sitting. So I guess I was very aware of the calorie count. The thing is, though, that they are delicious.\nAs someone who eats an entire bag of Reese'

### High p and top-k

In [ ]:
opt_lm.compile(sampler=keras_nlp.samplers.TopPSampler(p=0.8, k=5, seed=22))
opt_lm.generate("I recently ate", max_length=50)

'I recently ate an ice cream sandwich. It was a good ice cream sandwich.\nI recently ate an ice cream sandwich. It was a good ice cream sandwich.\nI recently ate an ice cream sandwich. It was a good ice cream sandwich'

## References

https://keras.io/api/keras_nlp/samplers/